In [ ]:
# Asked ChatGPT for a baseline and this is it
# The parameters are the below along with 70 landscape parameters with 
    #Index(['DateTime', 'lat', 'lon', 'wdir15', 't15', 'ws15', 'pres15', 'wdir150',
    #       't150', 'ws150', 'pres150', 'wdir250', 't250', 'ws250', 'pres250',
    #       'wdir500', 't500', 'ws500', 'pres500', 'f', 'fg', 'gust_factor'],
    #      dtype='object')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd, numpy as np, tensorflow as tf

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    return tf.reduce_mean(tf.abs((y_true-y_pred) / y_true)) * 100.0

In [ ]:
#df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-25ms-24hr-28-2-24.feather')
df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-w-landscape-full-25ms-24hr-28-2-24.feather')
df = df[df.f < df.fg]
df['gust_factor'] = df.fg / df.f

df = df.drop(['f', 'fg', 'fsdev', 'd', 'dsdev', 'longitude', 'latitude', 'X', 'Y', 'time', 'stod'], axis = 1)# + [f'Landscape_{i}' for i in range(70)], axis = 1)

df_unfolded = df.elevations.apply(pd.Series)
df = pd.concat([df, df_unfolded], axis = 1)

df = df.dropna()
df = df.reset_index(drop = True)

In [ ]:
from sklearn.decomposition import PCA

df_landscape_elevation = df.iloc[:, -70:]

df_landscape_elevation = (df_landscape_elevation - df_landscape_elevation.mean()) / df_landscape_elevation.std()

pca = PCA(n_components=3)
compressed_features = pca.fit_transform(df_landscape_elevation)

compressed_df = pd.DataFrame(data = compressed_features, columns = ['PC' + str(i) for i in range(3)])

df  = pd.concat([df, compressed_df], axis = 1)

In [ ]:
y = df.gust_factor
X = df.drop(['gust_factor'], axis = 1)
X = X.drop(['ws_15', 'ws_250', 'ws_500', 'wd_15', 'wd_250', 'wd_500', 'p_15', 'p_250', 'p_500', 't_15', 't_250', 't_500', 'N_01', 'N_12', 'landscape_points', 'elevations']
           + [i for i in range(70)], axis = 1)

# Changing the type of X,y so as to work with Tensorflow
X, y = X.values.astype(np.float32), y.values.astype(np.float32)
scaler = StandardScaler()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

In [ ]:
# Create a linear regression model
model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(units = 128, activation = 'relu', input_shape = (X_train.shape[1],)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units = 64, activation = 'relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units = 32, activation = 'relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units = 1, activation = 'linear')

    ]
) #LinearRegression()

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=2048, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=1024, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(units=8, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(units=1, activation='linear')
])

In [ ]:
model.compile(optimizer='adam', loss=mean_absolute_percentage_error)

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs = 5000, batch_size = 512, validation_data = (X_val, y_val))

In [ ]:
# Evaluate the model
mape = model.evaluate(X_test, y_test)

In [ ]:
y_predict = model.predict(X_test)

In [ ]:
print(f'Test MAPE: {mape}%')

In [ ]:
model.save('./saved_models/baselineLR.keras')